In [9]:
import os
import random
import numpy as np
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models



In [ ]:
# Configuration parameters
DATA_DIR = "/Users/bryangutierrez/Desktop/project3_ML"   
BATCH_SIZE = 8
NUM_EPOCHS = 15
LEARNING_RATE = 1e-4
VAL_SPLIT = 0.2
RANDOM_SEED = 42

GROUP_NUMBER = 16  
MODEL_PATH = f"Group_{GROUP_NUMBER}_CNN_FullModel.ph"


torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [ ]:
# Data transformations
train_transform = transforms.Compose([
    transforms.Resize((500, 500)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),

    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
# Validation transformations
val_transform = transforms.Compose([
    transforms.Resize((500, 500)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])



In [ ]:
# Load the full dataset
full_dataset = datasets.ImageFolder(root=DATA_DIR, transform=train_transform)

class_names = full_dataset.classes
print("Classes:", class_names)  

# Train/val split
dataset_size = len(full_dataset)
val_size = int(VAL_SPLIT * dataset_size)
train_size = dataset_size - val_size

train_dataset, val_dataset = random_split(
    full_dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(RANDOM_SEED)
)

# Update transforms for validation dataset
val_dataset.dataset.transform = val_transform

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Total images: {dataset_size}")
print(f"Train size: {train_size}, Val size: {val_size}")



Classes: ['not_official', 'official']
Total images: 25
Train size: 20, Val size: 5


In [ ]:
# Define the CNN model
class RazorbackCNN(nn.Module):
    """
    A small CNN for binary classification (official vs not official).
    Input: 3 x 500 x 500 images
    Output: 2 logits (for classes 0 and 1)
    """
    def __init__(self, num_classes=2):
        super(RazorbackCNN, self).__init__()

        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 500 -> 250

            # Block 2
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 250 -> 125

            # Block 3
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 125 -> 62/63

            # Block 4
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # ~62 -> ~31

           
            nn.AdaptiveAvgPool2d((4, 4))
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)  # 2 classes
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = RazorbackCNN(num_classes=2).to(device)
print(model)



RazorbackCNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): AdaptiveAvgPool2d(output_size=(4, 4))
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2048, out_features=256, bias=True)
    (2): ReLU(inplace=Tru

In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)



In [ ]:
# Training and evaluation functions
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total = 0

    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        running_corrects += torch.sum(preds == labels).item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = running_corrects / total
    return epoch_loss, epoch_acc


def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * images.size(0)
            running_corrects += torch.sum(preds == labels).item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = running_corrects / total
    return epoch_loss, epoch_acc



In [ ]:
# Training loop
best_val_acc = 0.0
best_model_state = None

for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)

    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} "
          f"|| Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = model.state_dict()

print(f"\nBest validation accuracy: {best_val_acc:.4f}")

# Load best model state
if best_model_state is not None:
    model.load_state_dict(best_model_state)



Epoch [1/15] Train Loss: 0.6916 | Train Acc: 0.6000 || Val Loss: 0.6929 | Val Acc: 0.6000
Epoch [2/15] Train Loss: 0.6804 | Train Acc: 0.6500 || Val Loss: 0.6921 | Val Acc: 0.6000
Epoch [3/15] Train Loss: 0.6913 | Train Acc: 0.6000 || Val Loss: 0.6914 | Val Acc: 0.6000
Epoch [4/15] Train Loss: 0.6721 | Train Acc: 0.6000 || Val Loss: 0.6927 | Val Acc: 0.6000
Epoch [5/15] Train Loss: 0.6720 | Train Acc: 0.6000 || Val Loss: 0.6949 | Val Acc: 0.6000
Epoch [6/15] Train Loss: 0.6488 | Train Acc: 0.6000 || Val Loss: 0.6994 | Val Acc: 0.6000
Epoch [7/15] Train Loss: 0.6640 | Train Acc: 0.6000 || Val Loss: 0.7077 | Val Acc: 0.6000
Epoch [8/15] Train Loss: 0.6579 | Train Acc: 0.6000 || Val Loss: 0.7203 | Val Acc: 0.6000
Epoch [9/15] Train Loss: 0.6461 | Train Acc: 0.6000 || Val Loss: 0.7355 | Val Acc: 0.6000
Epoch [10/15] Train Loss: 0.6370 | Train Acc: 0.6000 || Val Loss: 0.7493 | Val Acc: 0.6000
Epoch [11/15] Train Loss: 0.6243 | Train Acc: 0.6000 || Val Loss: 0.7633 | Val Acc: 0.6000
Epoch [1

In [ ]:
# Save the full model
torch.save(model, MODEL_PATH)
print(f"Saved full model to: {MODEL_PATH}")


RuntimeError: File Group_16_CNN_FullModel.ph cannot be opened.